 # AI Community @ Семинар № 4
 ## Деревья решений  
 <img src="./data/CART_tree_titanic_survivors.png" alt="Drawing" style="height: 300px;"/>
 ### Важная теория
 Энтропия.  
 Мера "однорости" данных  
 Низкая энтропия означает, что данные однородные  
 Высокая - что данные разнообразные  
 $$H(S)=\sum_{x \in X}{-p(x)\log_2 p(x)}$$
 $S$ - данные  
 $X$ - множество возможных значений данных  
 $p(x)$ - частота $x$ в $S$
 
 Пример:  
 Пусть в данных 0 встретился 1 раз, 1 встретилась 99 раз. Тогда энтропия равна
 $$H(S) = - 0.01 \cdot \log_2{0.01} - 0.99 \cdot \log_2{0.99} = 0.0808$$

In [6]:
import numpy as np
from scipy.stats import entropy
entropy([0.01, 0.99], base=2)

0.080793135895911181

Information gain (рус. Выигрыш от ветвления, прирост информации)
$$IG(A,S) = H(S) - \sum_{t \in T} p(t)H(t)$$  
$T$ – множество подмножеств, полученных разбиением по признаку A  
$p(t)$ – доля элементов с признаком $t$

 ### Алгоритм ID3
 1. Найти признак с наибольшим IG
 2. Разделить множество объектов по этому признаку.
 3. Повторить для каждого подмножества, если его энтропия не равна нулю  

Почитать ещё: http://www.saedsayad.com/decision_tree.htm

 ### Логистическая регрессия на деревьях

Берём рукописные цифры - 4 и 7

In [250]:
from sklearn import datasets
digits = datasets.load_digits()
X = digits.data[(digits.target == 4) | (digits.target == 7)]
y = digits.target[(digits.target == 4) | (digits.target == 7)]

Делим на train и test

In [251]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Обучаем два дерева высоты 1 - одно для верхней половины изображения, другое для второй

In [252]:
from sklearn.tree import DecisionTreeClassifier
classifier1 = DecisionTreeClassifier(criterion='entropy', max_depth=1)
classifier1.fit(X_train[:, :32], y_train)
classifier2 = DecisionTreeClassifier(criterion='gini', max_depth=1)
classifier2.fit(X_train[:, 32:], y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

Точность деревьев по отдельности:

In [253]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, classifier1.predict(X_test[:, :32])))
print(accuracy_score(y_test, classifier2.predict(X_test[:, 32:])))

0.902777777778
0.861111111111


Соберем выходы с деревьев в лог. регрессию:

In [254]:
from sklearn.linear_model import LogisticRegression
X_agg = np.vstack([classifier1.predict(X_train[:,:32]), classifier2.predict(X_train[:,32:])]).T
lclassifier = LogisticRegression()
lclassifier.fit(X_agg, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Смотрим результат:

In [255]:
c1_preds = classifier1.predict(X_test[:, :32])
c2_preds = classifier2.predict(X_test[:, 32:])
X_test_agg = np.vstack([c1_preds, c2_preds]).T
preds = lclassifier.predict(X_test_agg)
print(accuracy_score(y_test, preds))

0.916666666667


Смотрим веса деревьев:

In [256]:
lclassifier.coef_

array([[ 0.71998839,  0.56366117]])

 ### Бустинг
 
>*Ну ты строишь дерево, грубо говоря, возможно я ошибаюсь в этой штуке, потом ты смотришь там среднюю ошибку или что-то такое, короче, ну смысл был в том, что ты берешь это значение...*
<div style="text-align: right"> Восточная мудрость </div>

1. Строим простой классификатор
2. Считаем его ошибку (с учётом весов элементов выборки)
3. Находим вес классификатора (исходя из ошибки)
4. Обновляем веса элементов выборки с учётом ошибки и веса классификатора
5. Повторяем $n$ раз
6. Строем взвешенный классификатор

### AdaBoost
Дано: $(x_{1},y_{1}),\ldots,(x_{m},y_{m})$ , $x_{i} \in X,\, y_{i} \in Y = \{-1, +1\}$

$D_{1}(i) = \frac{1}{m}, i=1,\ldots,m.$

Для $t = 1,\ldots,T$:

* Находим классификатор $h_{t} : X \to \{-1,+1\}$; $h_{t} = \arg \min_{h_{j} \in \mathcal{H}} \epsilon_{j}$, где $ \epsilon_{j} = \sum_{i=1}^{m} D_{t}(i)[y_i \ne h_{j}(x_{i})]$
* Если $\epsilon_{t} \geqslant 0.5$, то останавливаемся.
* Выбираем $\alpha_{t} \in \mathbf{R}$, обычно $\alpha_{t}=\cfrac{1}{2}\textrm{ln}\cfrac{1-\epsilon_{t}}{\epsilon_{t}}$ где $\epsilon_{t}$ взвешенная ошибка классификатора $h_{t}$.
* Обновляем:
$D_{t+1}(i) = D_{t}(i) \, e^{- \alpha_{t} y_{i} h_{t}(x_{i})}$<br />
* Нормализуем $D_{t+1}(i)$

Строим результирующий классификатор:

$H(x) = \textrm{sign}\left( \sum_{t=1}^{T} \alpha_{t}h_{t}(x)\right)$

Красивые картинки для линейной регрессии тут: http://cmp.felk.cvut.cz/~sochmj1/adaboost_talk.pdf